# Theano basics

### Expressions

In [1]:
# Import numpy
import numpy as np
# Import theano and theano.tensor
import theano, theano.tensor as T
# Import Lasagne floatX utility function for ensuring that data is of the correct type
from lasagne.utils import floatX

# Create some test data to play with
a = floatX(np.arange(10.0).reshape((2,5)))
b = floatX(np.arange(10.0,20.0).reshape((5,2)))

# Load the data into Theano as shared data (note the optional name parameter)
a_t = theano.shared(a, name='a')
b_t = theano.shared(b)

# Use the Theano equivalent of numpy.dot() to get the dot product
ab_t = T.dot(a_t, b_t)

# Evaluate to get the result
print(ab_t.eval())

[[ 160.  170.]
 [ 510.  545.]]


Using gpu device 0: GRID K520 (CNMeM is enabled with initial size: 25.0% of memory, cuDNN 5004)


### Accessing and modifying shared data
Lets use `set_value` to change the values of the shared data and re-evaluate:

In [2]:
# Seeded random number generator
rng = np.random.RandomState(12345)

# Change the values of the shared data
a_t.set_value(floatX(rng.uniform(low=0, high=1, size=(2,5))))
b_t.set_value(floatX(rng.uniform(low=0, high=1, size=(5,2))))

# Print the new values then evaluate the previously built expression and print:
print(a_t.get_value())
print(b_t.get_value())
print(ab_t.eval())

[[ 0.92961609  0.31637555  0.18391882  0.20456028  0.567725  ]
 [ 0.5955447   0.96451449  0.65317708  0.74890661  0.65356988]]
[[ 0.74771482  0.96130675]
 [ 0.0083883   0.10644437]
 [ 0.2987037   0.65641117]
 [ 0.80981255  0.87217593]
 [ 0.96464759  0.72368532]]
[[ 1.46598887  1.63731575]
 [ 1.8854332   2.23007822]]


### Variables and functions

In [3]:
# Seeded random number generator
rng = np.random.RandomState(12345)

# Create a 2D variable; use the matrix constructor
x = T.matrix('x')

# Weights and biases of 5-channel input, 2-channel output
# linear model as shared data
W = theano.shared(floatX(rng.normal(0.25, size=(5,2))))
b = theano.shared(floatX(np.zeros((2,))))

# Linear model
linear = T.dot(x, W) + b[None,:]

# Evaluation function
eval_linear = theano.function([x], linear)

# Test it:
data_x = floatX(rng.uniform(0, 1, (10,5)))
print(data_x)
print(eval_linear(data_x))

[[ 0.74771482  0.96130675  0.0083883   0.10644437  0.2987037 ]
 [ 0.65641117  0.80981255  0.87217593  0.96464759  0.72368532]
 [ 0.64247531  0.7174536   0.467599    0.32558468  0.4396446 ]
 [ 0.72968906  0.99401456  0.67687368  0.79082251  0.17091426]
 [ 0.02684928  0.80037022  0.90372252  0.02467621  0.49174732]
 [ 0.52625519  0.59636599  0.05195754  0.89508951  0.72826618]
 [ 0.81835002  0.50022274  0.81018943  0.09596852  0.21895005]
 [ 0.25871906  0.46810576  0.45937321  0.70950979  0.17805301]
 [ 0.53144991  0.16774222  0.76881391  0.92817056  0.60949367]
 [ 0.1501835   0.48962671  0.37734497  0.8486014   0.91109723]]
[[ 0.13432555  0.76851845]
 [ 2.81232309  3.2601366 ]
 [ 1.43153954  1.84846342]
 [ 1.71037161  2.01665854]
 [ 2.29757953  2.00904632]
 [ 1.02733064  1.85243356]
 [ 1.953511    2.15374279]
 [ 1.32819879  1.44413781]
 [ 2.62176204  3.00520468]
 [ 1.93041217  2.39455056]]


### Gradient and updates

In [8]:
import sys

# Target variable; the target value
y = T.matrix('y')
# Learning rate variable
lr = T.scalar('learning_rate')

# Loss expression; mean squared error
loss = ((linear - y)**2).mean()

# Gradient of loss with respect to W and b
d_grad_d_W = T.grad(loss, wrt=W)
d_grad_d_b = T.grad(loss, wrt=b)

# Stochastic gradient descent; update x to new value
# Represent as dictionary mapping target to new value
updates = {W: W - d_grad_d_W * lr,
           b: b - d_grad_d_b * lr}

# Training function: given values for x and y, return
# the loss and apply updates
train_linear = theano.function([x, y, lr], loss, updates=updates)

# Test it
# Create target data, then print it
# The target will be a linear combination of the components of x plus
# some noise
factors = rng.normal(size=(5,2))
data_y = floatX(np.dot(data_x, factors) + rng.uniform(-0.1, 0.1, (10,2)))
print(data_y)
# 1000 iterations of SGD; the loss should decrease
for i in xrange(1000):
    l = train_linear(data_x, data_y, 0.01)
    sys.stdout.write('{} '.format(l))
    sys.stdout.flush()
sys.stdout.write('\n')
sys.stdout.flush()
# Evaluate the model
print(eval_linear(data_x))
print(data_y)
    

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: UserWarning: The parameter 'updates' of theano.function() expects an OrderedDict, got <type 'dict'>. Using a standard dictionary here results in non-deterministic behavior. You should use an OrderedDict if you are using Python 2.7 (theano.compat.OrderedDict for older python), or use a list of (shared, update) pairs. Do not just convert your dictionary to this type before the call as the conversion will still be non-deterministic.


[[-1.36257768 -0.65531474]
 [-0.78711748  0.8707363 ]
 [-1.19494736 -0.0079437 ]
 [-0.76314706 -0.38118801]
 [-0.83168006  0.08415516]
 [-0.35930121  1.19500184]
 [-1.72171664 -0.53175819]
 [ 0.10672021  0.41907474]
 [-0.59528166  1.30471754]
 [-0.10103182  1.80680192]]
2.15233039856 2.07839131355 2.00809979439 1.94127118587 1.87773060799 1.8173122406 1.75985753536 1.70521771908 1.65324997902 1.60381948948 1.55679774284 1.51206362247 1.46950137615 1.42900133133 1.39045929909 1.35377669334 1.31885933876 1.28561854362 1.25396943092 1.22383141518 1.1951290369 1.16778922081 1.14174389839 1.11692714691 1.09327757359 1.0707359314 1.04924654961 1.0287566185 1.00921571255 0.990575969219 0.97279214859 0.955821037292 0.939621925354 0.924156010151 0.909386336803 0.89527797699 0.881797671318 0.868913948536 0.856596767902 0.844817817211 0.833550155163 0.822768032551 0.812447190285 0.802564501762 0.793098092079 0.784027040005 0.77533185482 0.76699334383 0.758994102478 0.751316964626 0.743946075439 0